In [2]:
from openai import OpenAI
import os, toml
secrets = toml.load("../.streamlit/secrets.toml")
os.environ['OPENAI_API_KEY'] = secrets['OPEN_AI_KEY']
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
)

In [3]:
assistant = client.beta.assistants.create(
    name="BitGo RAG",
    instructions="You are an FAQ assistant for BitGo, always look at the attached files for all questions you are asked.  If you are looking at API requests make sure you have the full correct path. If you do not know the answer just say "I do not know the answer". When you are answering the questions do the following. Step 1. Think through the meaning of the question and break out if the person is wanting to know about API information, navigating the system, general information. The do the search with file_search. Step 2. Based on what you have from Step 1 you should answer the question by running a file_search function addressing each area they are interested in output them in a structured format.",
    tools=[{"type": "file_search"}],
    model="gpt-4-turbo"
)
assistant

Assistant(id='asst_do1pGfTj6rFPowBh7d0EtZpN', created_at=1699410542, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt-4-1106-preview', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])

In [4]:
# Create a thread
thread = client.beta.threads.create()
thread.id

'thread_n3xS8WjoEdzXVf8NPMTbRVbi'

In [5]:
prompt = "I need to solve the equation `3x + 11 = 14`. Can you help me?"
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)
message

ThreadMessage(id='msg_MWA07EToVkTyeA2FeOdZsBvD', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699410597, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_n3xS8WjoEdzXVf8NPMTbRVbi')

In [6]:
# List messages in the thread
thread_messages = client.beta.threads.messages.list(thread_id=thread.id)
print(thread_messages.data)

[ThreadMessage(id='msg_MWA07EToVkTyeA2FeOdZsBvD', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699410597, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_n3xS8WjoEdzXVf8NPMTbRVbi')]


In [7]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account."
    )
run

Run(id='run_hB9F5UWvWISwMCOrat6s831t', assistant_id='asst_do1pGfTj6rFPowBh7d0EtZpN', cancelled_at=None, completed_at=None, created_at=1699410948, expires_at=1699411548, failed_at=None, file_ids=[], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_n3xS8WjoEdzXVf8NPMTbRVbi', tools=[ToolAssistantToolsCode(type='code_interpreter')])

In [8]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
run

Run(id='run_hB9F5UWvWISwMCOrat6s831t', assistant_id='asst_do1pGfTj6rFPowBh7d0EtZpN', cancelled_at=None, completed_at=1699410955, created_at=1699410948, expires_at=None, failed_at=None, file_ids=[], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1699410948, status='completed', thread_id='thread_n3xS8WjoEdzXVf8NPMTbRVbi', tools=[ToolAssistantToolsCode(type='code_interpreter')])

In [19]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
for m in messages.data:
    print(m.role)
    print(m.content[0].text.value)
    print('--')

assistant
The solution to the equation \(3x + 11 = 14\) is \(x = 1\).
--
assistant
Absolutely, Jane Doe. To solve the equation \(3x + 11 = 14\), we need to find the value of \(x\) that makes this equation true. The steps to solve it are:

1. Subtract 11 from both sides of the equation to isolate the term with \(x\) on one side.
2. Divide by 3 to solve for \(x\).

Let me do the math for you:
--
user
I need to solve the equation `3x + 11 = 14`. Can you help me?
--
